In [1]:
import pandas as pd
import numpy as np
import os
import shutil
from pathlib import Path
import nltk
import string
import json
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')
import pickle


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
if not os.path.exists("dataset_copied"):
    shutil.copytree("dataset_new","dataset_copied")
    print("Copied Dataset Created !!!")  
else:
    print("Copied Dataset Exists !!!")

print("Length of Original Dataset :",len(os.listdir("dataset_new")))    
print("Length of Copied Dataset :",len(os.listdir("dataset_copied")))    

Copied Dataset Exists !!!
Length of Original Dataset : 1133
Length of Copied Dataset : 1133


In [3]:
all_filenames = []
for count, filename in enumerate(os.listdir("dataset_copied")):
    all_filenames.append(filename)
    f = open("dataset_copied/"+filename,"r",encoding='iso-8859-15')
    txt = f.read()
    f.close()
    # txt = Path("dataset_copied/"+filename).read_text()
    txt = txt.lower()
    f = open("dataset_copied/"+filename,"w",encoding='iso-8859-15')
    f.write(txt)
    f.close()
print("LowerCase Done !!!")
print("Length of Copied Dataset :",len(os.listdir("dataset_copied")))

LowerCase Done !!!
Length of Copied Dataset : 1133


In [4]:
all_filenames.sort()
name_to_id = {}
id_to_name = {}
k = 1
for i in all_filenames:
    name_to_id[i] = k
    id_to_name[k] = i
    k += 1

f = open("id_to_name.txt","wb")
pickle.dump(id_to_name,f)
f.close()

f = open("name_to_id.txt","wb")
pickle.dump(name_to_id,f)
f.close()

In [5]:
tokenizer = RegexpTokenizer(r'\w+')
id_to_tokens = {}

for filename in name_to_id:
    f = open("dataset_copied/"+filename,"r",encoding='iso-8859-15')
    txt = f.read()
    txt = txt.translate(str.maketrans(string.punctuation, ' '*len(string.punctuation)))
    tokens = tokenizer.tokenize(txt)
    tokens_without_sw = [word for word in tokens if not word in stopwords.words('english')]
    tokens = tokens_without_sw
    id_to_tokens[name_to_id[filename]]=tokens

f = open("id_to_tokens.txt","wb")
pickle.dump(id_to_tokens,f)
f.close()

In [6]:
f = open("id_to_tokens.txt","rb")
txt = f.read()
f.close()
id_to_tokens = pickle.loads(txt)

In [7]:
all_terms_without_id = []
all_terms_with_id = []
unique_terms_plus_doc_freq = {}


for i in id_to_tokens:
    tokens_of_i = id_to_tokens[i]
    for j in tokens_of_i:
        all_terms_without_id.append(j)
        all_terms_with_id.append((j,i))

        if j in unique_terms_plus_doc_freq:
            unique_terms_plus_doc_freq[j]+=1
        else:
            unique_terms_plus_doc_freq[j]=1
        

print(len(all_terms_without_id))
print(len(all_terms_with_id))
print(len(unique_terms_plus_doc_freq))

1165365
1165365
71038


In [8]:
all_terms_with_id.sort()
all_terms_without_id.sort()
items = unique_terms_plus_doc_freq.items()
unique_terms_plus_doc_freq = sorted(items)
unique_terms_plus_doc_freq = dict(unique_terms_plus_doc_freq)


In [9]:
terms_to_termplusfreq = {}

for i in unique_terms_plus_doc_freq:
    terms_to_termplusfreq[i] = (i,unique_terms_plus_doc_freq[i])

In [10]:
def get_postings(term,all_terms_with_id):
    postings_list = []
    for i in all_terms_with_id:
        if term in i:
            postings_list.append(i[1])
    return postings_list

In [11]:
inverted_index = {}

for i in unique_terms_plus_doc_freq:
    inverted_index[i]=[]

for i in id_to_tokens:
    tokens_of_i = id_to_tokens[i]
    for j in tokens_of_i:
        inverted_index[j].append(i)

for i in inverted_index:
    inverted_index[i] = list(set(inverted_index[i]))
    inverted_index[i].sort()

In [12]:
f = open("inverted_index.txt","wb")
pickle.dump(inverted_index,f)
f.close()

In [13]:
positional_index = {}

for i in inverted_index:
    positional_postings_list = {}
    positional_index[i] = positional_postings_list
    postings_list = inverted_index[i]
    for doc_id in postings_list:
        tokens = np.array(id_to_tokens[doc_id]) 
        positional_postings_list[doc_id]=list(np.where(tokens == i)[0])

In [14]:
f = open("positional_index.txt","wb")
pickle.dump(positional_index,f)
f.close()

In [15]:
f = open("positional_index.txt","rb")
txt = f.read()
f.close()
positional_index = pickle.loads(txt)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=04d84d63-ff1d-4672-9f78-d034a2868658' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>